# 00 - Initialize NER/Span segmentation ground-truth using a LLM

The aim of this notebook is to quickly initialise a ground-truth to further train a NER model without annotating the data by hand from scratch. The initial annotationis performed with a LLM (using Ollama). After a manual post-correction step, the annotations can be used to create a dataset to fine-tune Camembert-NER model (stored in Hugging-Face).

**You can skip this notebook. Download the NER data applied to taxpayers on [Zenodo](https://zenodo.org/10.5281/zenodo.15423885)**

Some informations about the named entities : 
* Name (person name or company)
* Firstnames
* Address
* Activity
* Title
* Family status

In [4]:
import pandas as pd
from ollama import chat
from pydantic import BaseModel
import time
import json
import regex

In [10]:
from formats import Taxpayer, TaxpayersList

In [5]:
ROOT = "/home/STual/DAN-cadastre"

## 1. Load the data

In [6]:
#Import entities
ENTITIES_CSV = ROOT + "/data/NER/entities.csv" #Available on Zenodo
df = pd.read_csv(ENTITIES_CSV)
display(df)

,element_uuid,address,plot_number,taxpayer,taxpayer_number,nature,former_plot_number,former_nature
0,33278fbe-9845-49a9-9fc0-00faf13c0484,Port à l'anglais,95,Charte Etienne,NaN,terre,NaN,NaN
1,58dfd0f9-62e9-4f95-b17d-922f3ce150d2,§,§,Crispin désiré augustin,NaN,§,NaN,NaN
2,c648b66a-1cd1-4a66-9b48-10429ea2f08d,§,§,chichérit françois auguste,NaN,§,NaN,NaN
3,93b4fad5-d5dd-4763-a56d-a35fc089c11d,§,§,C↑ie↓ parisienne du gaz,NaN,§,NaN,NaN
4,b3ad0224-1d5a-42b1-a41d-13531c23907b,§,§,Coquette léon et Coquette Louis→alfred,NaN,§,NaN,NaN
...,...,...,...,...,...,...,...,...
3221,97b9d7aa-e38b-4032-b263-2ac2433bb850,§,307,idem,167,maison b↑t↓→et cour,220p,Ø
3222,0829167d-6ff5-4ecd-bfd5-a30b7c40b2aa,§,308,idem,107,maison,220p,Ø
3223,5698c83e-8192-48f4-8542-7613edd5d528,le petit parc,16↑bis↓,Pierlot banquier à Paris,NaN,Gazon d'agt,NaN,NaN
3224,4b02c685-a8a2-4547-8002-09a8c6ac4fd4,le bois des champs,17,Delamare Mathurin Bourgeois,NaN,Bois,NaN,NaN


### 1.1 Filter the Taxpayer column

In [7]:
# Remove the value which don't describe a taxpayer
taxpayers_df = df[df['taxpayer'] != '§']
taxpayers_df = taxpayers_df[taxpayers_df['taxpayer'] != 'id']
taxpayers_df = taxpayers_df[taxpayers_df['taxpayer'] != 'idem']

## The following values have been treated by hand :( (should be uncomment in case of a new run)
#taxpayers_df = taxpayers_df[taxpayers_df['taxpayer'] != 'Id']
#taxpayers_df = taxpayers_df[taxpayers_df['taxpayer'] != 'Idem']
#taxpayers_df = taxpayers_df[taxpayers_df['taxpayer'] != 'Le même']
#taxpayers_df = taxpayers_df[taxpayers_df['taxpayer'] != 'Les mêmes']
#taxpayers_df = taxpayers_df[taxpayers_df['taxpayer'] != '\']

In [8]:
len(taxpayers_df)

2843

### 1.2 Set LLM parameters and prompt

In [9]:
# Roles
USER = 'user' #you
ASSISTANT = 'assistant' #the LLM
MODEL = "llama3.1"

In [11]:
EXAMPLES_LIST = [["Prudhomme",Taxpayer(name="Prudhomme")],
                ["Société anonyme du Comptoir Central de l'Est",Taxpayer(name="Société anonyme du Comptoir Central de l'Est")],
                ["Germay à Paris",Taxpayer(name="Germay", address=["Paris"])],
                ["Barbaroux quincailler à Paris",Taxpayer(name="Barbaroux",activity=["quincailler"],address=["Paris"])],
                ["Costy, Jn Bte Tailleur de Pierre à Villeneuve Leroy",Taxpayer(name="Costy", firstnames="Jn Bte",activity=["Tailleur de Pierre"],address=["Villeneuve Leroy"])],
                ["Besnet Joseph, Henri - 6/8 Rue Camille Desmoulins 19 Rue Guichard", Taxpayer(name="Besnet", firstnames="Joseph, Henri", address=["6/8 Rue Camille Desmoulins", "19 Rue Guichard"])],
                ["Tellier Catherine fille majeure à Ablon", Taxpayer(name="Tellier", firstnames="Catherine", familystatus=["fille majeure"], address=["Ablon"])],
                ["Pravel Louis Ve né Gerbuisson",Taxpayer(name="Pravel",firstnames="Louis",familystatus=["Ve"],birthname="Gerbuisson")],
                ["Commune d'Ablon",Taxpayer(name="Commune d'Ablon")]
]

In [12]:
PROMPT_CONTEXT = """You are a senior researcher specialized in digital humanities and linguistic. 
Your goal is to structure short texts describing taxpayers extracted from 19th century registers. More precisly, you have to create spans of these texts which are categorized as named entities described bellow.

### Entities

- **name**: The last name of a person or the name of a company.
- **firstnames**: One or more first names of the individual.
- **address**: An address or any other spatial entity associated with the individual.
- **activity"**: The profession, occupation, or work associated with the individual.
- **title**: Title of the individual such as 'M', 'Mme', 'Mademoiselle', 'Monseigneur', 'General', or 'Prince'.
- **familystatus**: Mentions of family or marital status of the individual such as 'Père', 'Veuve', 'Fille', 'Fils', or 'Héritier'.
- **birthname** : Name of an individual (in most of this case a woman) before its wedding

## Task description (hard constraints)
1. You can have one (more rarely 2 or 3) taxpayers described in a text.
2. You have to preserve original text strings and case.
3. The outputs have to be structure has the json described in the examples. 
4. If an entity type is not present in the input, it should be represented with an empty string (`''`) for singular entities or an empty list (`[]`) for plural entities.

### Examples:
Here are examples of inputs and outputs:
"""

In [13]:
PROMPT_EXAMPLES = ""
for elem in EXAMPLES_LIST:
    PROMPT_EXAMPLES += "\n**Input:** " + elem[0] + ""
    PROMPT_EXAMPLES += "\n**Output:** " + str(elem[1].model_dump())+"\n"

FULL_PROMPT = PROMPT_CONTEXT + PROMPT_EXAMPLES + "Please provide the response in the form of a list of JSON structured as in the example. It should begin with [ and end with ]."
print(FULL_PROMPT)

You are a senior researcher specialized in digital humanities and linguistic. 
Your goal is to structure short texts describing taxpayers extracted from 19th century registers. More precisly, you have to create spans of these texts which are categorized as named entities described bellow.

### Entities

- **name**: The last name of a person or the name of a company.
- **firstnames**: One or more first names of the individual.
- **address**: An address or any other spatial entity associated with the individual.
- **activity"**: The profession, occupation, or work associated with the individual.
- **title**: Title of the individual such as 'M', 'Mme', 'Mademoiselle', 'Monseigneur', 'General', or 'Prince'.
- **familystatus**: Mentions of family or marital status of the individual such as 'Père', 'Veuve', 'Fille', 'Fils', or 'Héritier'.
- **birthname** : Name of an individual (in most of this case a woman) before its wedding

## Task description (hard constraints)
1. You can have one (more

In [14]:
FULL_PROMPT = PROMPT_CONTEXT + PROMPT_EXAMPLES
print(FULL_PROMPT)

You are a senior researcher specialized in digital humanities and linguistic. 
Your goal is to structure short texts describing taxpayers extracted from 19th century registers. More precisly, you have to create spans of these texts which are categorized as named entities described bellow.

### Entities

- **name**: The last name of a person or the name of a company.
- **firstnames**: One or more first names of the individual.
- **address**: An address or any other spatial entity associated with the individual.
- **activity"**: The profession, occupation, or work associated with the individual.
- **title**: Title of the individual such as 'M', 'Mme', 'Mademoiselle', 'Monseigneur', 'General', or 'Prince'.
- **familystatus**: Mentions of family or marital status of the individual such as 'Père', 'Veuve', 'Fille', 'Fils', or 'Héritier'.
- **birthname** : Name of an individual (in most of this case a woman) before its wedding

## Task description (hard constraints)
1. You can have one (more

In [29]:
def named_entities(role, prompt, model):
    # Call the `chat` function with the specified model, format, and parameters.
    response = chat(
        model=model,
        format=TaxpayersList.model_json_schema(),
        messages=[
            {
                'role': role,
                'content': prompt
            },
        ]
    )
    # Validate and parse the response JSON into an AnimalList object.
    taxpayers_data = TaxpayersList.model_validate_json(response.message.content)
    return response

### 1.3 Execute the LLM

In [42]:
# Main block to execute the script.
results = []
if __name__ == "__main__":
    for i in range(600,700):
        # Path to the image to be analyzed.
        uuid = taxpayers_df.iloc[i]["element_uuid"]
        taxpayer_str = taxpayers_df.iloc[i]["taxpayer"]

        try:
            print("###########################################################")
            # Print an initial message before starting the analysis.
            print(f"\nAnalyzing a taxpayer : {taxpayer_str}")
        
            # Call the function to analyze the image and get the results.
            result = named_entities(ASSISTANT, FULL_PROMPT + "**Input:**" + taxpayer_str + " **Output:**" , MODEL)
    
            print(result.message.content)
            results.append({"element_uuid":uuid,"transcription":taxpayer_str,"entities_json":json.loads(result.message.content)})
            time.sleep(0.1)
        except:
            print("ERROR")
            print(uuid)

###########################################################

Analyzing a taxpayer : Desgranges p↑re↓→maire d'Ormesson
{ "taxpayers": [{ "name": "Desgranges", "firstnames": "", "activity": ["maire"], "address": ["d'Ormesson"], "title": [], "familystatus": ["p\u2192maire"], "birthname": "" }] }
###########################################################

Analyzing a taxpayer : Belleman Jean Marie→maçon à Berauié
{ "taxpayers": [ { "name": "Belleman", "firstnames": "Jean Marie", "activity": ["maçon"], "address": ["Berauié"], "title": [], "familystatus": [], "birthname": "" } ] }
###########################################################

Analyzing a taxpayer : Le Prince de Neuchâtel→×Vauguyon→Bourgeois, maire±
{"taxpayers":[{"name": "Le Prince de Neuch\u00e2tel", "firstnames": "", "activity": [], "address": [], "title": ["Prince"], "familystatus": [], "birthname": ""}, {"name": "\u00c0 Vauguyon", "firstnames": "", "activity": [], "address": [], "title": [], "familystatus": [], "birthname

In [43]:
with open('taxpayers_all.json', 'w',encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

**A post-correction-by-hand step of taxpayers.json is required here !!!**

### 1.4 Convert JSON to Brat format

In [15]:
import json
with open('taxpayers_all.json', 'r',encoding='utf-8') as file:
    results_f = json.load(file)

In [16]:
from formats import create_brat_entity_T

RES_PATH = ROOT + "/data/NER/NER_dataset"

for r in results_f:
    element_uuid = r['element_uuid']
    transcription = r['transcription']
    
    #Create files to store the results
    txt = open(RES_PATH + "/" + element_uuid + ".txt","w")
    txt.write(transcription)
    ann = open(RES_PATH + "/" + element_uuid + ".ann","w")
    
    #Create a counter for entities
    counter = 0
    for e in r['entities_json']['taxpayers']:
        keys = e.keys()
        for k in keys:
            entity_value = e[k]
            try:
                if isinstance(entity_value,str) and len(entity_value) != 0:
                    if transcription == entity_value:
                        brat = create_brat_entity_T(counter,k,0,len(transcription),transcription)
                        ann.write(brat)
                        counter +=1
                    else:
                        if '(' in entity_value and ')' in entity_value:
                            entity_value_r = entity_value.replace('(','\(').replace(')','\)')
                            reg = regex.search(entity_value_r,transcription)
                        elif '(' in entity_value:
                            entity_value_r = entity_value.replace('(','\(')
                            reg = regex.search(entity_value_r,transcription)
                        elif ')' in entity_value:
                            entity_value_r = entity_value.replace(')','\)')
                            reg = regex.search(entity_value_r,transcription)
                        else:
                            reg = regex.search(entity_value,transcription)
                        span = reg.span(0)
                        if span[1] - span[0] != 0:
                            brat = create_brat_entity_T(counter,k,span[0],span[1],reg.group(0))
                            ann.write(brat)
                            counter +=1
                elif isinstance(entity_value,list) and len(entity_value) != 0:
                    for part in entity_value:
                        reg = regex.search(part,transcription)
                        span = reg.span(0)
                        if span[1] - span[0] != 0:
                            brat = create_brat_entity_T(counter,k,span[0],span[1],reg.group(0))
                            ann.write(brat)
                            counter +=1
            except:
                print(element_uuid)
                print(transcription)

T0	name 0 6	Rameau

T1	firstnames 7 17	J↑n↓ louis

T2	familystatus 18 22	fils

T3	name 0 6	Rameau

T4	firstnames 7 17	J↑n↓ louis

T5	familystatus 23 28	veuve

T0	name 0 9	Vigoureux

T1	firstnames 10 16	f↑ois↓

T0	name 0 5	Royer

T1	firstnames 6 21	Jean louis marc

T0	name 0 6	Véron,

T1	firstnames 7 18	J↑n↓ J↑ques

T2	activity 20 28	Vig↑ron↓

T3	address 31 41	chenevière

T0	name 0 6	Gachet

T1	firstnames 7 28	p↑re↓ (dit le→dragon)

T2	activity 29 36	Vig↑on↓

T3	address 39 43	Sucy

T0	name 0 10	Bourdillat

T1	firstnames 11 15	J↑n↓

T0	name 0 5	Leroy

T1	firstnames 6 12	Gilles

T0	name 0 6	Rémoud

T1	firstnames 7 14	Charles

T0	name 0 6	Cochet

T1	firstnames 7 11	J↑n↓

T0	name 0 7	Pigeron

T0	name 0 5	Leroy

T1	firstnames 6 10	J↑n↓

T0	name 0 6	Collet

T1	firstnames 7 12	marie

T0	name 0 5	Baron

T1	firstnames 6 11	N↑as↓

T0	name 0 6	Pillet

T0	name 0 8	Mignière

T1	firstnames 9 15	Claude

T0	name 0 7	Pigeron

T0	name 5 11	Corday

T1	name 13 21	hospices

T0	name 0 5	Leroy

T1	firstnames 

## Unit tests

In [17]:
t = "Chaillou Louis frédéric"
reg = regex.search("Louis frédéric",t)
print(reg)

<regex.Match object; span=(9, 23), match='Louis frédéric'>


In [18]:
import glob

len(glob.glob(RES_PATH + '/*.txt'))

483